In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('./../data/BBC/bbc_data.csv', sep=';', index_col=0)

In [33]:
df

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,spacied_syno_l,processed_syno_l,processed_syno_l_one_string,k_means
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247,"['music', 'from-the-archives']",Katie Derham hosts continued live coverage fro...,"['Katie', 'Derham', 'host', 'continue', 'live'...",Katie Derham host continue live coverage Royal...,11
1,1,Port,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037,['music'],Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,"['Bidh', 'Druthag', 'Bheag', 'Eile', 'tarraing...",Bidh Druthag Bheag Eile tarraing ri chèile pìo...,7
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,10 Feb 2022,611,"['music', 'entertainment']",Tom Holland and Greg James cold call celebrity...,"['Tom', 'Holland', 'Greg', 'James', 'cold', 'c...",Tom Holland Greg James cold celebrity friend f...,10
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ceiliúradh na Féile Pádraig",John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,False,17 Mar 2017,3538,['music'],John Toal and Pauline Scanlon introduce an eve...,"['John', 'Toal', 'Pauline', 'Scanlon', 'introd...",John Toal Pauline Scanlon introduce evening mu...,7
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Songs of Praise, Edinburgh",Claire McCollum explores Edinburgh and shares ...,Claire McCollum goes on a Christian heritage t...,Claire McCollum explores Edinburgh and discove...,Music,bbc_one,False,1:15pm 20 Feb 2022,2044,"['music', 'lifestyle']",Claire McCollum explores Edinburgh and discove...,"['Claire', 'McCollum', 'explore', 'Edinburgh',...",Claire McCollum explore Edinburgh discover ric...,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777,3777,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Models: Street to Catwalk, S...",Kyra’s late arrival puts her modelling career ...,Kyra jeopardises her modelling career by turni...,How will Kyra cope after a train malfunction m...,Documentary,bbc_three,Contains some strong language and some upsetti...,8 Aug 2021,1843,['documentaries'],How will Kyra cope after a train malfunction m...,"['Kyra', 'cope', 'train', 'malfunction', 'make...",Kyra cope train malfunction make hour crucial ...,9
3789,3789,Death On The I-95,Returning to the US to find out what happened ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Death On The I-95",Returning to the US to find out what happened ...,Following a report on the growing problem of o...,Drug overdose is now the main cause of death f...,Documentary,bbc_three,Contains some strong language and upsetting sc...,24 Oct 2018,342

In [34]:
import spacy
from tqdm.auto import tqdm

def spacy_text(texts):
    # the pipeline’s config.cfg tells spaCy to use the language "en" and the pipeline
    # ["tok2vec", "tagger", "parser", "ner", "attribute_ruler", "lemmatizer"].
    # spaCy will then initialize spacy.lang.en.English, and create each pipeline component and add it to the processing pipeline.
    # It’ll then load in the model data from the data directory and return the modified Language class for you to use as the nlp object.
    # https://spacy.io/usage/processing-pipelines
    nlp = spacy.load("en_core_web_sm")

    # extract all values from the text column
    text_list = []
    if isinstance(texts, str):
        text_list.append(texts)
    else:
        text_list = texts.tolist()

    # When you call nlp on a text, spaCy first tokenizes the text to produce a Doc object.
    # We disable {parser = Assign dependency labels, ner = Detect and label named entities} out of reasons I frankly don't know but follow as it is stated in the manual.
    disable = ["ner", "parser"]

    # Here the input (text) is transformed into spacy.tokens.doc.Doc.
    # In addition with tqdm the progress of the transformation is shown.
    return [text for text in tqdm(nlp.pipe(text_list, n_process=1, disable=disable), total=len(text_list))]

In [35]:
df['spacied_syno_l'] = spacy_text(df['syno_l'])

  0%|          | 0/2326 [00:00<?, ?it/s]

In [36]:
from enum import Enum
from nltk.stem.porter import PorterStemmer
"""
Pre processes spacied values.

This function is able to
    - remove punctuation
        - is always removed
    - remove stopwords
        - removed when @param rm_stopwords = True
    - filter for specific token.pos (see SpacyPos for all possibilities)
        - does not filter for specific tokens if param is []
    - lemmatize the spacied values
        - lemmatize the values if lemma = True
    - stems the spacied values
        - stems the values if lemma = False
    - always lower cases all values

@return a list of all preprocessed tokenized values
"""
def pre_process(spacied_values, pos, base_form = 'lemma', rm_stopwords = True, min_length_word=2, token=False):
    result = []
    for text in spacied_values:
        # removes all unnecessary text like punctuations and stop words
        processed_text = rm_unnecessary_text(text, rm_stopwords)
        # filters for specific words
        if pos:
            processed_text = filter_specific_pos(processed_text, pos)
        # transforms the base form depending on the base_form input
        if base_form == 'lemma':
            processed_text = lemmatization(processed_text)
        elif base_form == 'stem':
                processed_text = stemming(processed_text)
        else:
            processed_text = text_values(processed_text)
        # Lowers all the tokens
        processed_text = remove_words_with_length(min_length_word, processed_text)
        if token:
            result.append(processed_text)
        else:
            result.append(lower(processed_text))

    return result

def lower(values):
    return [token.lower() for token in values]

def rm_unnecessary_text(values, rm_stopwords):
    if rm_stopwords:
        return [token for token in values if not token.is_punct and not token.is_stop]
    else:
        return [token for token in values if not token.is_punct]

def filter_specific_pos(values, pos):
    pos_values = []
    for val in pos:
        if not isinstance(val, SpacyPos):
            raise TypeError('pos must be an instance of SpacyPos')
        pos_values.append(val.value)
    return [token for token in values if token.pos_ in pos_values]

def text_values(values):
    return [token.text for token in values]

def lemmatization(values):
    return [token.lemma_ for token in values]

def stemming(values):
    stemmer = PorterStemmer()
    return [stemmer.stem(token.text) for token in values]

def remove_words_with_length(min_length, values):
    return [token for token in values if len(token) >= min_length]

"""
I made a SpacyEnum so I assure that I do not get an exception when I accidentally misspelled a POS Tag.

URLs to remember:
- https://spacy.io/api/token
- https://universaldependencies.org/u/pos/
- https://web.archive.org/web/20190206204307/https://www.clips.uantwerpen.be/pages/mbsp-tags
"""
class SpacyPos(Enum):
    ADJ = 'ADJ'      # adjective                     e.g. nice, easy
    ADP = 'ADP'      # adposition
    ADV = 'ADV'      # adverb                        e.g. extremely, loudly, hard
    AUX = 'AUX'      # auxiliary                     e.g. may, should
    CCONJ = 'CCONJ'  # coordinating conjunction      e.g. and, or, but
    DET = 'DET'      # determiner                    e.g. the, a, these
    INTJ = 'INTJ'    # interjection                  e.g. oh, oops, gosh
    NOUN = 'NOUN'    # noun                          e.g. tiger, chair, laughter
    NUM = 'NUM'      # numeral
    PART = 'PART'    # particle                      e.g. about, off, up
    PRON = 'PRON'    # pronoun                       e.g. my, your, our
    PROPN = 'PROPN'  # proper noun                   e.g. Germany, God, Alice
    PUNCT = 'PUNCT'  # punctuation                   e.g. .;?*
    SCONJ = 'SCONJ'  # subordinating conjunction     e.g. whether or not
    SYM = 'SYM'      # symbol                        e.g. %
    VERB = 'VERB'    # verb                          e.g. think, eat, talk
    X = 'X'          # other

In [37]:
pos_napv = [SpacyPos.NOUN, SpacyPos.ADJ, SpacyPos.PROPN, SpacyPos.VERB]
df['processed_syno_l']  = pre_process(df['spacied_syno_l'], pos=pos_napv, base_form='lemma', rm_stopwords=True, token=True)

In [38]:
df['processed_syno_l']

0       [Katie, Derham, host, continue, live, coverage...
1       [Bidh, Druthag, Bheag, Eile, tarraing, ri, chè...
2       [Tom, Holland, Greg, James, cold, celebrity, f...
3       [John, Toal, Pauline, Scanlon, introduce, even...
4       [Claire, McCollum, explore, Edinburgh, discove...
                              ...                        
3777    [Kyra, cope, train, malfunction, make, hour, c...
3789    [drug, main, cause, death, Americans, age, lik...
3793    [music, industry, react, surge, outrage, death...
3797    [Sliabh, Eachtaí, tá, Cérill, ag, déanamh, réi...
3800    [Tim, Bergling, regular, teenager, live, small...
Name: processed_syno_l, Length: 2326, dtype: object

In [39]:
df

,index,title,description,image,keywords,syno_s,syno_m,syno_l,category,channel,language,release_date,duration_sec,topic,spacied_syno_l,processed_syno_l,processed_syno_l_one_string,k_means
0,0,BBC Proms,Australian tenor Stuart Skelton is joined by L...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, BBC Proms, 2021: Last Night ...",Australian tenor Stuart Skelton is joined by L...,Australian tenor Stuart Skelton is joined by L...,Katie Derham hosts continued live coverage fro...,Music,bbc_radio_three,False,9pm 11 Sep 2021,5247,"['music', 'from-the-archives']","(Katie, Derham, hosts, continued, live, covera...","[Katie, Derham, host, continue, live, coverage...",Katie Derham host continue live coverage Royal...,11
1,1,Port,"Previously unseen music from the series Port, ...",https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Port, Series 5: Episode 4","Previously unseen music from the series Port, ...",Julie Fowlis a’ lìbhrigeadh ceòl bhon t-sreath...,Bidh Druthag Bheag Eile a’ tarraing ri chèile ...,Music,bbc_alba,False,9 Jul 2020,1037,['music'],"(Bidh, Druthag, Bheag, Eile, a, ’, tarraing, r...","[Bidh, Druthag, Bheag, Eile, tarraing, ri, chè...",Bidh Druthag Bheag Eile tarraing ri chèile pìo...,7
2,2,"Sit Down, Stand Up with Greg James",Tom Holland and Greg James cold call celebrity...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Sit Down, Stand Up with Greg...",Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Tom Holland and Greg James cold call celebrity...,Entertainment,bbc_radio_one,False,10 Feb 2022,611,"['music', 'entertainment']","(Tom, Holland, and, Greg, James, cold, call, c...","[Tom, Holland, Greg, James, cold, celebrity, f...",Tom Holland Greg James cold celebrity friend f...,10
3,3,Ceiliúradh na Féile Pádraig,John Toal and Pauline Scanlon introduce an eve...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Ceiliúradh na Féile Pádraig",John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,John Toal and Pauline Scanlon introduce an eve...,Music,bbc_two_northern_ireland_digital,False,17 Mar 2017,3538,['music'],"(John, Toal, and, Pauline, Scanlon, introduce,...","[John, Toal, Pauline, Scanlon, introduce, even...",John Toal Pauline Scanlon introduce evening mu...,7
4,4,Songs of Praise,Claire McCollum explores Edinburgh and shares ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Songs of Praise, Edinburgh",Claire McCollum explores Edinburgh and shares ...,Claire McCollum goes on a Christian heritage t...,Claire McCollum explores Edinburgh and discove...,Music,bbc_one,False,1:15pm 20 Feb 2022,2044,"['music', 'lifestyle']","(Claire, McCollum, explores, Edinburgh, and, d...","[Claire, McCollum, explore, Edinburgh, discove...",Claire McCollum explore Edinburgh discover ric...,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3777,3777,Models: Street to Catwalk,Kyra’s late arrival puts her modelling career ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Models: Street to Catwalk, S...",Kyra’s late arrival puts her modelling career ...,Kyra jeopardises her modelling career by turni...,How will Kyra cope after a train malfunction m...,Documentary,bbc_three,Contains some strong language and some upsetti...,8 Aug 2021,1843,['documentaries'],"(How, will, Kyra, cope, after, a, train, malfu...","[Kyra, cope, train, malfunction, make, hour, c...",Kyra cope train malfunction make hour crucial ...,9
3789,3789,Death On The I-95,Returning to the US to find out what happened ...,https://ichef.bbci.co.uk/images/ic/{recipe}/p0...,"BBC, iPlayer, TV, Death On The I-95",Returning to the US to find out what happened ...,Following a report on the growing problem of o...,Drug overdose is now the main cause of death f...,Documentary,bbc_three,Contains some strong language and upsetting sc...,24 

In [40]:
df['processed_syno_l_one_string'] = df['processed_syno_l'].apply(lambda x: ' '.join(x))

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import cluster

vectorizer = TfidfVectorizer(min_df=3, max_df=0.80, norm='l2')
X = vectorizer.fit_transform(df['processed_syno_l_one_string'])
tf_idf = pd.DataFrame(data = X.toarray(), columns=vectorizer.get_feature_names())
tf_idf

/Users/hagenaadfock/miniforge3/envs/tf/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10th,150th,18th,19,1930,1940,1950,1960,1970,1980,...,às,èideann,òg,òga,òrain,òran,óg,ùr,ùra,ùrlar
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.057735,0.067856,0.0,0.0,0.0,0.067856,0.135711,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [48]:
clusters = 10
kmeanModel = KMeans(n_clusters=clusters, init='k-means++', max_iter=3000, random_state=0)
mod = kmeanModel.fit_transform(tf_idf)
df['k_means'] = kmeanModel.predict(tf_idf)

In [49]:
order_centroids = kmeanModel.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
dict = []
for i in range(clusters):
  print('%d' % i, sep='', end=', '),
  for ind in order_centroids[i, :20]:
    print(terms[ind], sep='', end=', ')
  print('')

0, agus, air, ann, tha, ag, bho, na, alba, gu, sa, seo, gaelic, mar, aig, bidh, toirt, le, ach, iad, mu, 
1, new, war, run, year, world, force, time, life, story, old, final, find, fight, president, big, tom, power, face, relationship, bring, 
2, day, coverage, man, highlight, question, wedding, present, scottish, final, find, parliament, look, february, live, night, league, minister, cup, face, wales, 
3, christmas, festive, special, tree, santa, carol, sing, family, choir, favourite, seasonal, feast, little, celebrate, gift, star, mary, find, nigella, come, 
4, friend, find, join, family, get, team, good, try, help, comedy, win, time, series, head, come, host, make, year, take, party, 
5, news, late, weather, sport, mp, debate, political, england, interview, bbc, east, international, west, national, midlands, feature, south, yorkshire, north, norris, 
6, people, life, work, year, world, artist, art, change, look, time, meet, film, help, woman, way, explore, live, new, find, black, 
7

In [51]:
df[df['k_means'] == 1]['title'].head(20)

61                            Snow Patrol at Ward Park 3
125                                               Rewind
161    New Power Generation: Black Music Legends of t...
181                    Everything - The Real Thing Story
194                   The 80s - Music’s Greatest Decade?
206                 Castles: Britain's Fortified History
211                                                 Ex-S
212                                   Handmade in Bolton
240                                              Omnibus
262                                      Building Sights
320                                    Where We Live Now
327              A Very British Murder with Lucy Worsley
340                                        Inside Cinema
345                             The Infinite Monkey Cage
358                           Fred Dibnah's Age of Steam
366              Arnold Schwarzenegger on Climate Change
383                                   The World About Us
396                 Neutrino: H

In [45]:
extra_genres = {0: 'Scottish Gaelic', 1: 'Slice of Life', 2: 'Challenging', 3: ''}

In [46]:
# df.to_csv('../data/BBC/bbc_data.csv', sep=';')